In [ ]:
# This is my child class with the parent class to be able to iteratively backtest 
# strategy. In this file I have added the ability to backtest an SMA strategy using 
# the iterative backtest OOP

In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [45]:
class IterativeBase():

    def __init__(self,symbol,start,end,amount, use_spread = True):
        self.symbol = symbol
        self.start = start
        self.end = end
        self.initial_balance = amount
        self.current_balance = amount
        self.units = 0
        self.trades = 0
        self.position = 0
        self.use_spread = use_spread
        self.get_data()

    def get_data(self):
        raw = pd.read_csv("detailed.csv", parse_dates = ["time"],index_col = "time")
        raw = raw.loc[self.start:self.end]
        raw["returns"] = np.log(raw.price/raw.price.shift(1))
        self.data = raw

    def plot_data(self, cols = None):
        if cols is None:
            cols = "price"
        self.data[cols].plot(figsize = (12,8), title = self.symbol)

    def get_values(self, bar):
        date = str(self.data.index[bar].date())
        price = round(self.data.price.iloc[bar], 5)
        spread = round(self.data.spread.iloc[bar], 5)
        return date,price,spread

    def print_current_balance(self, bar):
        date, price, spread = self.get_values(bar)
        print("{} | Current Balance: {}".format(date, round(self.current_balance, 5)))

    def buy_instrument(self, bar, units = None, amount = None):
        date, price, spread = self.get_values(bar)
        if self.use_spread:
            price += spread/2
        if amount is not None:
            units = int(amount/price)
        self.current_balance -= units*price
        self.units += units
        self.trades += 1
        print("{} | Buying {} for {}".format(date,units,round(price, 5)))

    def sell_instrument(self, bar, units = None, amount = None):
        date, price, spread = self.get_values(bar)
        if self.use_spread:
            price -= spread/2
        if amount is not None:
            units = int(amount/price)
        self.current_balance += units*price
        self.units -= units
        self.trades += 1
        print("{} | Selling {} for {}".format(date,units,round(price, 5)))

    def print_current_position_value(self, bar):
        date, price, spread = self.get_values(bar)
        cpv = self.units*price
        print("{} | Current Position Value = {}".format(date, round(cpv, 4)))

    def print_current_NAV(self, bar):
        date, price, spread = self.get_values(bar)
        nav = self.current_balance + self.units*price
        print("{} | Net Asset Value = {}".format(date, round(nav, 5)))

    def close_pos(self,bar):
        date, price, spread = self.get_values(bar)
        print(75 * "-")
        print("{} | +++CLOSING FINAL POSITION +++".format(date))
        self.current_balance += self.units * price
        self.current_balance -= (abs(self.units)*spread/2*self.use_spread)
        print("{} | closing position of {} for {}".format(date, self.units, price))
        self.units = 0
        self.trades +=1
        perf = (self.current_balance - self.initial_balance)/self.initial_balance * 100
        self.print_current_balance(bar)
        print("{} | net performance(%) = {}".format(date, round(perf, 4)))
        print("{} | number of trades executed = {}".format(date, self.trades))
        print(75 * "-")
    

In [46]:
class IterativeBackTest(IterativeBase):

    def go_long(self,bar, units = None, amount = None):
        if self.position == -1:
            self.buy_instrument(bar, units = -self.units)
        if units:
            self.buy_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.buy_instrument(bar, amount = amount)


    def go_short(self, bar, units = None, amount = None):
        if self.position == 1:
            self.sell_instrument(bar, units = self.units)
        if units:
            self.sell_instrument(bar, units = units)
        elif amount:
            if amount == "all":
                amount = self.current_balance
            self.sell_instrument(bar, amount = amount)

    def test_sma_strategy(self, SMA_S, SMA_L):
        stm = "Testing SMA Strategy | {} | SMA_S = {} & SMA_L = {}".format(self.symbol, SMA_S, SMA_L)
        print("-" * 75)
        print(stm)
        print("-" * 75)

        self.position = 0
        self.trades = 0
        self.current_balance = self.initial_balance
        self.get_data()

        self.data["SMA_S"] = self.data["price"].rolling(SMA_S).mean()
        self.data["SMA_L"] = self.data["price"].rolling(SMA_L).mean()
        self.data.dropna(inplace = True)

        for bar in range(len(self.data)-1):
            if self.data["SMA_S"].iloc[bar] > self.data["SMA_L"].iloc[bar]:
                if self.position in [0,-1]:
                    self.go_long(bar, amount = "all")
                    self.position = 1
            elif self.data["SMA_S"].iloc[bar] < self.data["SMA_L"].iloc[bar]:
                if self.position in [0,1]:
                    self.go_short(bar, amount = "all")
                    self.position = -1

        self.close_pos(bar+1)





In [47]:
bc = IterativeBackTest("EURUSD", "2006-12-31", "2020-06-30", 10000, use_spread = True)

In [48]:
bc.data

,price,spread,returns
time,,,
2006-12-31 22:00:00+00:00,1.31985,0.00100,NaN
2007-01-01 22:00:00+00:00,1.32734,0.00015,0.005659
2007-01-02 22:00:00+00:00,1.31688,0.00015,-0.007912
2007-01-03 22:00:00+00:00,1.30845,0.00015,-0.006422
2007-01-04 22:00:00+00:00,1.30025,0.00100,-0.006287
...,...,...,...
2020-06-23 21:00:00+00:00,1.12507,0.00030,-0.005151
2020-06-24 21:00:00+00:00,1.12180,0.00023,-0.002911
2020-06-25 21:00:00+00:00,1.12184,0.00041,0.000036


In [49]:
bc.test_sma_strategy(50,200)

---------------------------------------------------------------------------
Testing SMA Strategy | EURUSD | SMA_S = 50 & SMA_L = 200
---------------------------------------------------------------------------
2007-08-10 | Buying 7299 for 1.36989
2008-08-28 | Selling 7299 for 1.46685
2008-08-28 | Selling 7299 for 1.46685
2009-04-28 | Buying 7299 for 1.3276
2009-04-28 | Buying 8831 for 1.3276
2010-01-20 | Selling 8831 for 1.40833
2010-01-20 | Selling 8831 for 1.40833
2010-09-27 | Buying 8831 for 1.35862
2010-09-27 | Buying 9477 for 1.35862
2011-01-12 | Selling 9477 for 1.33626
2011-01-12 | Selling 9477 for 1.33626
2011-01-30 | Buying 9477 for 1.36946
2011-01-30 | Buying 9017 for 1.36946
2011-09-10 | Selling 9017 for 1.35825
2011-09-10 | Selling 9017 for 1.35825
2012-10-08 | Buying 9017 for 1.28857
2012-10-08 | Buying 9992 for 1.28857
2013-04-15 | Selling 9992 for 1.31755
2013-04-15 | Selling 9992 for 1.31755
2013-08-04 | Buying 9992 for 1.32592
2013-08-04 | Buying 9866 for 1.32592
2014-0